# 1ª Lista de Exercícios

In [2]:
# Funções auxiliares

def describe_problem(problem):
    for variable in problem.variables():
        print(variable.name, "=", round(variable.varValue, 2))
    print(problem.objective.name, "=", round(problem.objective.value(), 2))

## Execicio 1

Maximizar
<br>
$Z = 3x_1 + 6x_2$

Restrições

$9x_1 + 8x_2 \leq 72$
<br>$x_2 \leq 6$
<br>$-5x_1 + 4x_2 \leq 20$
<br>$2x_1 - 4x_2 \leq 20$
<br>$x_1, x_2 \geq 0$

Implementando em python usando pulp


In [3]:
from pulp import LpVariable, LpProblem, LpMaximize

problem1 = LpProblem("Execicio1", LpMaximize)

x1 = LpVariable("x1", lowBound=0, cat='Continuous')
x2 = LpVariable("x2", lowBound=0, cat='Continuous')

# Função objetivo
problem1 += 3 * x1 + 6 * x2
# Restrições
problem1 += 9 * x1 + 8 * x2 <= 72
problem1 += x2 <= 6
problem1 += -5 * x1 + 4 * x2 <= 20
problem1 += 2 * x1 - 4 * x2 <= 20

problem1.solve()

describe_problem(problem1)

x1 = 2.67
x2 = 6.0
OBJ = 44.0


## Execicio 2

Maximizar
<br>
$Z = 10x_1 + 2x_2$ <br>

Restricoes

$10x_1 + 4x_2 \leq 40$
<br>$8x_1 + 2x_2 \geq 0$ <br>
$x_2 \leq 6$ <br>
$x_1 - 3x_2 \leq 0$ <br>
$x_1, x_2 \geq 0$

Solução

In [4]:
problem2 = LpProblem("Execicio2", LpMaximize)

x1 = LpVariable("x1", lowBound=0, cat='Continuous')
x2 = LpVariable("x2", lowBound=0, cat='Continuous')

# Função objetivo
problem2 += 10 * x1 + 2 * x2

# Restrições
problem2 += 10 * x1 + 4 * x2 <= 40
problem2 += 8 * x1 + 2 * x2 >= 0
problem2 += x2 <= 6
problem2 += x1 - 3 * x2 <= 0

problem2.solve()

describe_problem(problem2)

x1 = 3.53
x2 = 1.18
OBJ = 37.65


## Execicio 3

Uma  pequena  manufatura  produz  dois  modelos,  Standard  e  Luxo,  de  um  certo  produto.

Cada unidade do modelo Standard requer 2 horas de lixação e 1 hora de polimento.
Cada unidade do modelo Luxo exige 2 horas de lixação e 3 horas de polimento.

$tempo_{standard} = 2 + 1$ horas
$tempo_{luxo} = 2 + 3$ horas

A fábrica dispõe de 2 lixadoras e 3  polidoras,  cada  qual  trabalhando  40  horas  semanais.

$horas_{lixador} = 2 \cdot 40$ semanais
$horas_{polidora} = 3 \cdot 40$ semanais

As  margens  de  lucro  são  $24  e  $32, respectivamente, para  cada unidade  Standard e  Luxo.
Não  existem restrições  de demanda  para nenhum  dos  modelos.  Elabore  um  modelo  de  programação  linear  que  nos  permita  calcular  a produção semanal que maximize a margem total de lucro do fabricante.